In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=78bc947fdea59efa84bb9cda071dabc74afda5fa2e55a8bf08b685e6ef767ff8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
! pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
#First Path
path = "cleaned_water_data.csv"
school_df = spark.read.option("header",'True').option('delimiter', ',').csv(path)

#Second Path
pathtwo = "cleaned_blood_data.csv"
lead_df= spark.read.option("header",'True').option('delimiter', ',').csv(pathtwo)


Preview Both Data Sets

In [38]:
#First Data Frame School Water
school_df.show(5)

+------+-----+--------------------+--------------------+-------------------------------------+---------------------------------------+----------------+----------------+--------------------+
|County|  Zip|              School|Type of Organization|Any Building with Lead-Free Plumbing?|Number of Outlets that Require Sampling|Outlets ≤ 15 ppb|Outlets > 15 ppb|     County Location|
+------+-----+--------------------+--------------------+-------------------------------------+---------------------------------------+----------------+----------------+--------------------+
|Albany|12009|         ALTAMONT ES|       Public School|                                   No|                                   63.0|            63.0|             0.0|(42.678066, -73.8...|
|Albany|12047|ABRAM LANSING SCHOOL|       Public School|                                   No|                                   74.0|            74.0|             0.0|(42.678066, -73.8...|
|Albany|12047|  BOGHT HILLS SCHOOL|       Public S

In [39]:
#Second DataFrame Lead levels in blood
lead_df.show(5)


+------+-----------+-----+----+-------------+-----+------------------+-----------+------------+----------+---------------------------+--------------+-------+--------------------+--------------------+
|County|County Code|  Zip|Year|Year of Birth|Tests|Less than 5 mcg/dL|5-10 mcg/dL|10-15 mcg/dL|15+ mcg/dL|Total Elevated Blood Levels|Rate per 1,000|Percent|   Zip Code Location|     County Location|
+------+-----------+-----+----+-------------+-----+------------------+-----------+------------+----------+---------------------------+--------------+-------+--------------------+--------------------+
|Albany|          1|12009|2020|         2019|   30|              30.0|        0.0|         0.0|       0.0|                        0.0|           0.0|    0.0|(42.697778, -74.0...|(42.588271, -73.9...|
|Albany|          1|12023|2020|         2019|    7|               7.0|        0.0|         0.0|       0.0|                        0.0|           0.0|    0.0|(42.60636, -74.1438)|(42.588271, -73.9...|


In [ ]:
#look at the statistics of plumbing outlets
school_df.select(['Outlets ≤ 15 ppb','Outlets > 15 ppb']).describe().show()

+-------+------------------+-----------------+
|summary|  Outlets ≤ 15 ppb| Outlets > 15 ppb|
+-------+------------------+-----------------+
|  count|              3366|             3366|
|   mean| 80.46850861556744|4.065359477124183|
| stddev|48.434493872706874|8.314144501152382|
|    min|               0.0|              0.0|
|    max|              99.0|             91.0|
+-------+------------------+-----------------+



In [ ]:
#look at the statistics of lead in blood levels of children
lead_df.select(['Less than 5 mcg/dL','5-10 mcg/dL','10-15 mcg/dL','15+ mcg/dL']).describe().show()

+-------+------------------+--------------------+------------+----------+
|summary|Less than 5 mcg/dL|         5-10 mcg/dL|10-15 mcg/dL|15+ mcg/dL|
+-------+------------------+--------------------+------------+----------+
|  count|              1817|                1817|        1817|      1817|
|   mean| 25.73913043478261|0.022564667033571822|         0.0|       0.0|
| stddev| 33.12551510551277|  0.3969007470413673|         0.0|       0.0|
|    min|              10.0|                 0.0|         0.0|       0.0|
|    max|              99.0|                 9.0|         0.0|       0.0|
+-------+------------------+--------------------+------------+----------+



# Schools in New York Water Data
First read in the data using pyspark, dropped columns that did not have testable data in them, then split the data into Training and testing sets to check the accuracy of the following question:
    Are there any schools in New york that have Lead Free Plumbing?

In [ ]:
#Drop specific columns that are not needed I created a variable, so it would copy and not edit the original dataframe
school=school_df.drop('County','County Location','Type of Organization','School')
school.show()

+-----+-------------------------------------+---------------------------------------+----------------+----------------+
|  Zip|Any Building with Lead-Free Plumbing?|Number of Outlets that Require Sampling|Outlets ≤ 15 ppb|Outlets > 15 ppb|
+-----+-------------------------------------+---------------------------------------+----------------+----------------+
|12009|                                   No|                                   63.0|            63.0|             0.0|
|12047|                                   No|                                   74.0|            74.0|             0.0|
|12047|                                   No|                                   81.0|            80.0|             1.0|
|12047|                                   No|                                   53.0|            53.0|             0.0|
|12047|                                   No|                                   86.0|            86.0|             0.0|
|12084|                                 

In [ ]:
#Select your features (columns)
school.columns

['Zip',
 'Any Building with Lead-Free Plumbing?',
 'Number of Outlets that Require Sampling',
 'Outlets ≤ 15 ppb',
 'Outlets > 15 ppb']

Split Train/Test School Data

In [31]:
#Index Categorical data from strings to int
indexer= StringIndexer(inputCol='Any Building with Lead-Free Plumbing?',outputCol='Any Building with Lead-Free Plumbing?_idx')

#Assign index values to strings
indexer = indexer.fit(school)

#Create column with index values
school = indexer.transform(school)

In [35]:
#Assembling Columns
assembler=VectorAssembler(inputCols=['Zip', 'Number of Outlets that Require Sampling','Outlets ≤ 15 ppb','Outlets > 15 ppb','Any Building with Lead-Free Plumbing?_idx'], outputCol='features')

# Consolidate predictor columns
schools_assembled = assembler.transform(school)

# Check the resulting column
schools_assembled.select('features','')


IllegalArgumentException: ignored

In [32]:
#Split into Training and testing sets in a 80:20 ratio
school_train, school_test=school.randomSplit([0.8,0.2],seed=42)

In [33]:
#Check that training set has around 80% of records
training_ratio=school_train.count()/school.count()
print(training_ratio)

0.8152109328579917


In [37]:
# Create the decision tree classifier instance
tree = DecisionTreeClassifier()
#fitting the model
school_tree_model = tree.fit(school_train)


In [37]:
# Create predictions for the testing data and take a look at the predictions
prediction = school_tree_model.transform(school_test)
prediction.select('label','prediction','probability').show()